In [1]:
import pandas as pd
import gender_guesser.detector as gender

### Funciones

In [2]:
def is_initial(name):
    return len(name) < 4 and '.' in name

### Definicion de input/output

In [3]:
# PROCESS = 'influenced'
PROCESS = 'influencer'

### Genderizacion

In [4]:
d = gender.Detector(case_sensitive=False)

In [5]:
DATA = '../data/'

In [6]:
DATA_SCRAPING = '../scrapping/outputs/authors_data/'

In [7]:
if PROCESS == 'influenced':
    influenced_authors = pd.read_csv(DATA_SCRAPING+'influenced_by_1_from_1.csv')
elif PROCESS == 'influencer':
    influenced_authors = pd.read_csv(DATA_SCRAPING+'influenced_authors_1_from_1.csv')
else:
    print('Falta definir variable PROCESS -go to the top-')    

In [8]:
influenced_authors.head()

,author_id,id,name,score
0,1726246,1748224,Rafail Ostrovsky,486
1,1726246,1752788,Dan Boneh,249
2,1726246,1679615,Michael T. Goodrich,246
3,1726246,145310230,Craig Gentry,238
4,1726246,1811683,Rafael Pass,230


In [9]:
len(influenced_authors)

129008

In [10]:
influenced_authors['first_name'] = influenced_authors.name.apply(lambda x: x.split()[0])

In [11]:
influenced_authors.head()

,author_id,id,name,score,first_name
0,1726246,1748224,Rafail Ostrovsky,486,Rafail
1,1726246,1752788,Dan Boneh,249,Dan
2,1726246,1679615,Michael T. Goodrich,246,Michael
3,1726246,145310230,Craig Gentry,238,Craig
4,1726246,1811683,Rafael Pass,230,Rafael


In [12]:
influenced_authors.sample(10)

,author_id,id,name,score,first_name
17335,28803996,1689241,Yanxi Liu,15,Yanxi
104905,1921515,1941805,Blazej Lewcio,19,Blazej
90387,2159263,145778478,Nils Stein,96,Nils
42430,3362957,2098502,Corrado Costa,72,Corrado
30598,49743071,145370903,Gabriele Guidi,41,Gabriele
72080,145254543,1703213,Pasi Fränti,28,Pasi
57329,2673376,66252329,Tommy E Cederholm,139,Tommy
111820,47489587,1747126,Thomas I. Strasser,41,Thomas
17563,1398099420,144159535,Alan C. Evans,103,Alan
80447,3331804,40543952,Qunjing Wang,12,Qunjing


## Creo 'first_name'

In [13]:
influenced_authors[influenced_authors['first_name'].apply(lambda x: len(x)<4)]

,author_id,id,name,score,first_name
1,1726246,1752788,Dan Boneh,249,Dan
9,145078811,1716421,Van Jacobson,225,Van
10,144066998,1752788,Dan Boneh,31,Dan
11,144066998,144467753,Ion Stoica,27,Ion
12,144066998,144473638,W. Keith Edwards,26,W.
...,...,...,...,...,...
128929,3196491,144470378,Yu Lei,70,Yu
128943,2747138,144286539,K. Gopakumar,98,K.
128944,2747138,144687982,K. Gopakumar,78,K.
128946,2747138,1937015,R. Sudharshan Kaarthik,45,R.


## Detecto iniciales en 'first_name' (no las descarto)

In [14]:
influenced_authors['is_initial'] = influenced_authors.first_name.apply(is_initial)
influenced_authors.is_initial.value_counts()

False    125935
True       3073
Name: is_initial, dtype: int64

In [15]:
influenced_authors.head()

,author_id,id,name,score,first_name,is_initial
0,1726246,1748224,Rafail Ostrovsky,486,Rafail,False
1,1726246,1752788,Dan Boneh,249,Dan,False
2,1726246,1679615,Michael T. Goodrich,246,Michael,False
3,1726246,145310230,Craig Gentry,238,Craig,False
4,1726246,1811683,Rafael Pass,230,Rafael,False


In [16]:
# influenced_authors = influenced_authors[~ influenced_authors.is_initial]

In [17]:
len(influenced_authors)

129008

## Genderizo archivo para id unicos

In [18]:
influenced_authors_uniques = influenced_authors[['id', 'first_name']].drop_duplicates()

In [19]:
len(influenced_authors_uniques)

64060

In [20]:
influenced_authors_uniques.head()

,id,first_name
0,1748224,Rafail
1,1752788,Dan
2,1679615,Michael
3,145310230,Craig
4,1811683,Rafael


In [21]:
influenced_authors_uniques['gender'] = influenced_authors_uniques.first_name.apply(lambda x: d.get_gender(x))

In [22]:
influenced_authors_uniques.head()

,id,first_name,gender
0,1748224,Rafail,male
1,1752788,Dan,male
2,1679615,Michael,male
3,145310230,Craig,male
4,1811683,Rafael,male


## Mergeo con archivo original

In [23]:
influenced_authors = influenced_authors.merge(influenced_authors_uniques, on=['id', 'first_name'], how='left')

In [24]:
influenced_authors[influenced_authors.gender == 'unknown']

,author_id,id,name,score,first_name,is_initial,gender
12,144066998,144473638,W. Keith Edwards,26,W.,True,unknown
33,40448872,1703441,Mihir Bellare,117,Mihir,False,unknown
35,35964168,144161012,Kannan Ramchandran,76,Kannan,False,unknown
37,35964168,1806989,Avideh Zakhor,64,Avideh,False,unknown
44,1764975,1687984,Shamkant B. Navathe,50,Shamkant,False,unknown
...,...,...,...,...,...,...,...
128960,91889051,3189258,Simira Papadopoulou,81,Simira,False,unknown
128968,3415759,51118223,Ulia Trifonova,3,Ulia,False,unknown
128979,145295484,145691986,P. Jonathon Phillips,320,P.,True,unknown
128981,145295484,50295995,Xiaoou Tang,242,Xiaoou,False,unknown


##  Corroboro que no haya nulls en 'gender'

In [25]:
influenced_authors[influenced_authors.gender.isnull()]

,author_id,id,name,score,first_name,is_initial,gender


## Muestreo de distribucion de genero

In [26]:
influenced_authors_groupped = influenced_authors.groupby('gender').count()
influenced_authors_groupped

,author_id,id,name,score,first_name,is_initial
gender,,,,,,
andy,5622,5622,5622,5622,5622,5622
female,15484,15484,15484,15484,15484,15484
male,87293,87293,87293,87293,87293,87293
mostly_female,1072,1072,1072,1072,1072,1072
mostly_male,2873,2873,2873,2873,2873,2873
unknown,16664,16664,16664,16664,16664,16664


In [27]:
influenced_authors_groupped['gender_rate'] = round((influenced_authors_groupped['author_id']*100)/(influenced_authors_groupped['author_id'].sum()),2)

In [28]:
influenced_authors_groupped = influenced_authors_groupped[['author_id', 'gender_rate']].rename(columns={'author_id':'count'})

In [29]:
influenced_authors_groupped

,count,gender_rate
gender,,
andy,5622,4.36
female,15484,12.00
male,87293,67.66
mostly_female,1072,0.83
mostly_male,2873,2.23
unknown,16664,12.92


## Preparo archivo a exportar

In [30]:
file_to_export = influenced_authors[['author_id', 'id', 'name', 'score', 'is_initial', 'gender']].rename(columns={'name':'full_name'})

## Exporto a csv

In [31]:
if PROCESS == 'influenced':
    file_name = 'influenced_by_author_id_name_gender.csv'
elif PROCESS == 'influencer':
    file_name = 'influencer_author_id_name_gender.csv'     
else:
    print('Falta definir variable PROCESS -go to the top-')
    

In [32]:
file_to_export.to_csv(DATA+file_name, index=False)
print("Se exportó archivo {file_name} en la ruta {path} para '{type_author} authors'".format(file_name=file_name, path=DATA, type_author=PROCESS)) 

Se exportó archivo influencer_author_id_name_gender.csv en la ruta ../data/ para 'influencer authors'


In [33]:
len(file_to_export)

129008